In [1]:
import numpy as np
import pandas_datareader as web
import datetime
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt

In [7]:
np.random.seed(42)

# Definisco i ticker che costituiscono il portafoglio, la data di inizio e la data di fine
tickers = ["EXV1.DE", "EXV2.DE", "EXV3.DE", "EXV5.DE", "EXV6.DE"]
start_date = "2014-06-06"
end_date = "2024-05-27"

# Scarico i dati per ciascun ticker (Adj Close e Volume)
stocks_data = yf.download(tickers, start=start_date, end=end_date, interval='1d')
stocks_data["Adj Close"].to_csv("stocks_data_adj_close.csv")

# Faccio il resample dei dati a cadenza settimanale (usando la chiusura del venerdì)
weekly_adj_close = stocks_data['Adj Close'].resample('W-FRI').last()
weekly_volume = stocks_data['Volume'].resample('W-FRI').sum()

# Calcolo il logaritmo dei rendimenti settimanali e della variazione di volume settimanale
log_returns = np.log(weekly_adj_close / weekly_adj_close.shift(1))
log_returns.dropna(inplace=True)
log_volume_change = np.log(weekly_volume / weekly_volume.shift(1))
log_volume_change.dropna(inplace=True)

# Unisco i rendimenti logaritmici e le variazioni logaritmiche del volume
features = log_returns.join(log_volume_change, lsuffix='_log_returns', rsuffix='_log_volume_change')

# Aggiungo colonne per anno, mese e settimana
features['Year'] = features.index.year
features['Month'] = features.index.month
features['Week'] = features.index.isocalendar().week

features

[*********************100%%**********************]  5 of 5 completed


In [3]:
np.random.seed(42)

# Definisco i ticker che costituiscono il portafoglio, la data di inizio e la data di fine
tickers = ["GC=F", "CL=F"]

# Scarico i dati per ciascun ticker (Adj Close e Volume)
gold_oil_data = yf.download(tickers, start=start_date, end=end_date, interval='1d')

# Faccio il resample dei dati a cadenza settimanale (usando la chiusura del venerdì)
weekly_adj_close = gold_oil_data['Adj Close'].resample('W-FRI').last()
weekly_volume = gold_oil_data['Volume'].resample('W-FRI').sum()

# Calcolo il logaritmo dei rendimenti settimanali e della variazione di volume settimanale
log_returns = np.log(weekly_adj_close / weekly_adj_close.shift(1))
log_returns.dropna(inplace=True)
log_volume_change = np.log(weekly_volume / weekly_volume.shift(1))
log_volume_change.dropna(inplace=True)

# Unisco i rendimenti logaritmici e le variazioni logaritmiche del volume
gold_oil_features = log_returns.join(log_volume_change, lsuffix='_log_returns', rsuffix='_log_volume_change')

# Aggiungo colonne per anno, mese e settimana
gold_oil_features['Year'] = features.index.year
gold_oil_features['Month'] = features.index.month
gold_oil_features['Week'] = features.index.isocalendar().week

# Unisco le features i dati macroeconomici (prezzo oro e petrolio)
features = pd.merge(features, gold_oil_features, on=['Year', 'Month', 'Week'], how='left')
features

[*********************100%%**********************]  2 of 2 completed


,EXV1.DE_log_returns,EXV2.DE_log_returns,EXV3.DE_log_returns,EXV5.DE_log_returns,EXV6.DE_log_returns,EXV1.DE_log_volume_change,EXV2.DE_log_volume_change,EXV3.DE_log_volume_change,EXV5.DE_log_volume_change,EXV6.DE_log_volume_change,Year,Month,Week,CL=F_log_returns,GC=F_log_returns,CL=F_log_volume_change,GC=F_log_volume_change
0,-0.011657,-0.001659,0.001065,-0.018531,-0.010386,1.322026,0.255979,4.518050,1.224688,3.175903,2014,6,24,0.040565,0.017104,1.857315,1.374413
1,-0.021527,-0.008005,0.007070,0.007492,0.020664,-1.940665,-0.685042,-4.943396,-1.810953,-1.004738,2014,6,25,0.003268,0.032823,-0.266786,0.356476
2,-0.031544,-0.005036,-0.021721,-0.020887,-0.008598,0.971962,1.039662,3.945714,1.650998,-0.381296,2014,6,26,-0.014273,0.002125,-0.048384,0.145546
3,0.012289,0.000673,0.024183,0.021269,0.052791,0.331784,-0.544316,-0.167457,-1.039380,0.828782,2014,7,27,-0.016016,0.001061,-0.087460,-0.852267
4,-0.040179,-0.030740,-0.036133,-0.039213,-0.023715,1.274162,0.929091,0.554028,0.796088,1.124818,2014,7,28,-0.031532,0.012494,0.348549,-1.425704
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515,0.037578,0.018244,0.047080,0.006671,0.035591,0.091663,-0.986098,-0.503742,0.831730,-0.597248,2024,4,17,0.008504,-0.026876,0.436641,-0.050638
516,-0.003028,-0.003214,-0.015364,-0.032372,0.006969,-0.448823,-0.025055,-0.384351,-0.211539,0.190596,2024,5,18,-0.070911,-0.015452,0.179813,-0.933427
517,0.029233,0.009612,0.041512,-0.003841,0.023621,0.446622,0.915176,0.054841,0.990475,-0.391248,2024,5,19,0.001919,0.029276,-0.194512,-1.567136
518,0.020404,0.033859,0.007399,0.013062,0.023384,-0.453543,0.626886,0.605748,-0.538852,-0.094406,2024,5,20,0.022740,0.018789,-0.212307,0.464813


In [4]:
# Carico i dati sui tassi di interesse dell'area Euro (fonte: https://fred.stlouisfed.org/series/IR3TIB01EZM156N) 
eu_interest_rates = pd.read_csv("eu_interest_rates.csv")
eu_interest_rates['Date'] = pd.to_datetime(eu_interest_rates['Date'])

# Carico i dati sull'indice armonizzato dei prezzi al consumo HCPI  (fonte: https://ec.europa.eu/eurostat/databrowser/view/prc_hicp_midx__custom_11923354/default/table?lang=en) 
eu_hcpi = pd.read_csv("eu_hcpi.csv")
eu_hcpi['Date'] = pd.to_datetime(eu_hcpi['Date'] + '-01')

# Unisco i tassi di interesse e il valore dell' indice HCPI in un unico DataFrame
interest_rates_and_hcpi = pd.merge(eu_interest_rates, eu_hcpi, on='Date', how='left')
interest_rates_and_hcpi["Date"] = pd.to_datetime(interest_rates_and_hcpi["Date"])
interest_rates_and_hcpi = interest_rates_and_hcpi[interest_rates_and_hcpi["Date"] >= pd.to_datetime('2014-05-01')]

# Aggiungo le colonne anno e mese
interest_rates_and_hcpi['Year'] = interest_rates_and_hcpi['Date'].dt.year
interest_rates_and_hcpi['Month'] = interest_rates_and_hcpi['Date'].dt.month

# Calcolo il logaritmo delle variazioni del HCPI e standardizzo i tassi di interesse
interest_rates_and_hcpi['log_change_eu_hcpi'] = np.log(interest_rates_and_hcpi['eu_hcpi'] / interest_rates_and_hcpi['eu_hcpi'].shift(1))
interest_rates_and_hcpi["eu_interest_rates_z_score"]= (eu_interest_rates["eu_interest_rates"] - eu_interest_rates["eu_interest_rates"].mean()) / eu_interest_rates["eu_interest_rates"].std()

# Rimuovo le colonne originali dei tassi di interesse e del hcpi e le righe con valori NaN
interest_rates_and_hcpi = interest_rates_and_hcpi.drop(columns=['eu_hcpi', 'eu_interest_rates'])
interest_rates_and_hcpi = interest_rates_and_hcpi.dropna()

# Unisco le features i dati macroeconomici (tassi di interesse area euro e tasso di inflazione area euro)
features = pd.merge(features, interest_rates_and_hcpi, on=['Year', 'Month'], how='left')
features = features.drop(columns=['Date', 'Year', 'Week'])
features

,EXV1.DE_log_returns,EXV2.DE_log_returns,EXV3.DE_log_returns,EXV5.DE_log_returns,EXV6.DE_log_returns,EXV1.DE_log_volume_change,EXV2.DE_log_volume_change,EXV3.DE_log_volume_change,EXV5.DE_log_volume_change,EXV6.DE_log_volume_change,Month,CL=F_log_returns,GC=F_log_returns,CL=F_log_volume_change,GC=F_log_volume_change,log_change_eu_hcpi,eu_interest_rates_z_score
0,-0.011657,-0.001659,0.001065,-0.018531,-0.010386,1.322026,0.255979,4.518050,1.224688,3.175903,6,0.040565,0.017104,1.857315,1.374413,0.000998,-0.057502
1,-0.021527,-0.008005,0.007070,0.007492,0.020664,-1.940665,-0.685042,-4.943396,-1.810953,-1.004738,6,0.003268,0.032823,-0.266786,0.356476,0.000998,-0.057502
2,-0.031544,-0.005036,-0.021721,-0.020887,-0.008598,0.971962,1.039662,3.945714,1.650998,-0.381296,6,-0.014273,0.002125,-0.048384,0.145546,0.000998,-0.057502
3,0.012289,0.000673,0.024183,0.021269,0.052791,0.331784,-0.544316,-0.167457,-1.039380,0.828782,7,-0.016016,0.001061,-0.087460,-0.852267,-0.005200,-0.083717
4,-0.040179,-0.030740,-0.036133,-0.039213,-0.023715,1.274162,0.929091,0.554028,0.796088,1.124818,7,-0.031532,0.012494,0.348549,-1.425704,-0.005200,-0.083717
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515,0.037578,0.018244,0.047080,0.006671,0.035591,0.091663,-0.986098,-0.503742,0.831730,-0.597248,4,0.008504,-0.026876,0.436641,-0.050638,0.005859,2.568643
516,-0.003028,-0.003214,-0.015364,-0.032372,0.006969,-0.448823,-0.025055,-0.384351,-0.211539,0.190596,5,-0.070911,-0.015452,0.179813,-0.933427,0.001843,2.516255
517,0.029233,0.009612,0.041512,-0.003841,0.023621,0.446622,0.915176,0.054841,0.990475,-0.391248,5,0.001919,0.029276,-0.194512,-1.567136,0.001843,2.516255
518,0.020404,0.033859,0.007399,0.013062,0.023384,-0.453543,0.626886,0.605748,-0.538852,-0.094406,5,0.022740,0.018789,-0.212307,0.464813,0.001843,2.516255


In [5]:
# A partire dalla colonna Month ottengo 12 variabili dummy
features = pd.get_dummies(features, columns=['Month'], prefix='Month')
# Converto colonne dummy da True/False a 0/1
dummy_columns = features.filter(like='Month_').columns
features[dummy_columns] = features[dummy_columns].astype(int)

features = features.dropna()

In [6]:
# Controllo la presenza di eventuali valori NaN
nan_counts = features.isna().sum()
print(nan_counts)

# Salvo le features in un csv
features.to_csv("features.csv", index= False)

EXV1.DE_log_returns          0
EXV2.DE_log_returns          0
EXV3.DE_log_returns          0
EXV5.DE_log_returns          0
EXV6.DE_log_returns          0
EXV1.DE_log_volume_change    0
EXV2.DE_log_volume_change    0
EXV3.DE_log_volume_change    0
EXV5.DE_log_volume_change    0
EXV6.DE_log_volume_change    0
CL=F_log_returns             0
GC=F_log_returns             0
CL=F_log_volume_change       0
GC=F_log_volume_change       0
log_change_eu_hcpi           0
eu_interest_rates_z_score    0
Month_1                      0
Month_2                      0
Month_3                      0
Month_4                      0
Month_5                      0
Month_6                      0
Month_7                      0
Month_8                      0
Month_9                      0
Month_10                     0
Month_11                     0
Month_12                     0
dtype: int64
